<h1>Programación matemática</h1>
<p>Este modelo genera las cantidades sugeridas a pedir un único proveedor, que puede atender a varias bodegas, cada una de ellas con su propio forecast de venta en función del forecast de venta.</p>
<p><strong>Conjuntos</strong>:</p>
<ul>
    <li>b: conjunto de bodegas</li>
    <li>t: conjunto de periodos</li>
</ul>
<p><strong>Parámetros</strong>:</p>
<ul>
    <li>$ HE $: costo unitario de mantener el producto en bodega. Surge del costo del producto y el costo de oportunidad por periodo</li>
    <li>$ CP $: costo de colocar una orden de abastecimiento para un sku totalizando las tres bodegas</li>
    <li>$ I_{bo} $: Inventario inicial en la bodega b</li>
    <li>$ SS_{b} $: Inventario de seguridad para el final del periodo. El modelo intentará mantener el cierre de inventario sobre esta cifra en on_hand</li>
    <li>$ LM $: Lote mínimo de producción, asegurará que la suma del pedido de las bodegas no esté por debajo de este valor.</li>
    <li>$ MPO $: Pedido mínimo: Asegura que no se sugieran pedidos de menos de este valor.</li>
    <li>$ F_{bt} $: Pronóstico de ventas del producto en la bodega b durante el periodo t</li>
    <li>$ A_{bt} $: Llegadas calculadas de producto a la bodega b durante el periodo t</li>
    <li>$ LT_{b} $: Lead time calculado entre las plantas y las bodegas</li>
</ul>
<p><strong>Variables</strong>:</p>
<ul>
    <li>$ X_{bt} $: Cantidad de producto a pedir para la bodega b durante el periodo t</li>
    <li>$ M_{bt} $: Binaria, 1 si se coloca el pedido mínimo (30 metros?) durante el periodo t para la bodega b; 0 en otr caso</li>
    <li>$ P_{t} $: Binary, 1 si se coloca un pedido desde cualquiera de las bodegas durante el periodo t. Esta se usará para garantizar el pedido mínimo para producción</li>
    <li>$ I_{bt} $: Variable de Estado, Cantidad de producto al cierre del periodo t en la bodega b</li>
</ul>
<p><strong>Función objetivo</strong>:</p>
<p>La siguiente expresión minimiza el costo total relevante compuesto por el costo total de colocar las órdenes de pedido y el costo de mantener el inventario:</p>
<p>$ min \sum_{bt}{CP*P_{t} + HE*X_{bt}} $ </p>
<p><strong>Sujeto a</strong>:</p>
<p><strong>Resticción de Inventario de seguridad:</strong> Evita que quede inventario por debajo del SS luego del leadtime</p>
<p>$ I_{bt} \geq {SS_{bt}} \forall{t > LT_{b}}$</p>
<p><strong>Flujos de inventario:</strong> Mantiene el balance de masa del inventario. Para el primer caso, se encarga de inicializar el inventario, en el segundo del flujo durante el lead time y, finalmente, el flujo cuando los pedidos colocados empiezan a llegar</p>
<p>$ I_{bt} = I_{bo} - F_{bt} + A_{bt} \forall{b}; t=0 $</p>
<p>$ I_{bt} = I_{bt} - F_{bt} + A_{bt} \forall b ; \forall 1 < t < LT $</p>
<p>$ I_{bt} = I_{bt} - F_{bt} + A_{bt} + X_{bt} \forall b ; \forall t \geq LT $</p>

In [1]:
from lp_supply.inventory_solvers import mono_product_multiperiod_multi_location
import numpy as np

In [2]:
np.sum([[10,0,0,0,0]])

10

In [3]:
pedidos, inventario, presupuesto, arrivals = mono_product_multiperiod_multi_location(holding_cost=0.0,
                                                                                     order_cost=0.0,
                                                                                     initial_inventory=[0.0, 0.0, 0.0],
                                                                                     safety_stock=[10,10,10],
                                                                                     min_batch_size=400,
                                                                                     min_po=50,
                                                                                     forecast=[[10,0,0,0,0],[0,10,0,0,0],[0,0,0,10,0]],
                                                                                     arrivals=[[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],
                                       leadtime=[1,3,1])

([[0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0]],
 [[0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0]],
 [[10, 0, 0, 0, 0], [0, 10, 0, 0, 0], [0, 0, 0, 10, 0]],
 [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]])